In [42]:
import numpy as np

def step(M,n,s,p,h,encoding):
    k=np.nanargmin(M)
    k_value=np.nanmin(M)
    i=k//n
    j=k-i*n
    s.append(s[i]+s[j])
    encoding[s[i]+s[j]]=k_value/2
    h.append('('+str(h[i])+':'+str(k_value/2-encoding[s[i]])+', '+str(h[j])+':'+str(k_value/2-encoding[s[j]])+')')
    s.pop(i)
    h.pop(i)
    s.pop(j-1)
    h.pop(j-1)
    uu=[]
    for r in range(0,n):
        if r!=i and r!=j:
            u=(M[min(i,r),max(i,r)]+M[min(j,r),max(j,r)])/2
            uu.append(u)
    uu.append(np.nan)
    M=np.delete(M, j, 0)
    M=np.delete(M, i, 0)
    M=np.delete(M, j, 1)
    M=np.delete(M, i, 1)
    M=np.insert(M, n-2, np.nan, axis=0)
    M=np.insert(M, n-2, uu, axis=1)
    return M, n-1,s,k_value,h,encoding

def main(s,M):    
    encoding = {}
    for l in s:
        encoding[l]=0.0   
    n=len(s)
    h=s.copy()
    p=0
    while n>1:
        M,n,s,p,h,encoding=step(M,n,s,p,h,encoding)
    

def dist(a,b,M,s):
    a_ind=s.index(a)
    b_ind=s.index(b)
    return M[min(a_ind,b_ind),max(a_ind,b_ind)]
    
def s_dist(a,b,M,s):
    a_ind=s.index(a)
    t=0
    for w in s:
        if a!=w and w!=b:
            t=t+dist(a,w,M,s)
    t=t/(len(s)-2)
    return t


def k(a,b,M,s):
    t=dist(a,b,M,s)-s_dist(a,b,M,s)-s_dist(b,a,M,s)
    return t
            
    
def ch(M,s):
    t=0
    mm=[]
    for a in s:
        for b in s:
            if a!=b:
                m=k(a,b,M,s)
                mm.append(m)
                if m<t:
                    t=m
                    x,y=a,b
    #print(mm)
    return x,y
                
#x,y=ch()

def step2(M,s,x,y,encoding,deep):
    n=len(s)
    i=min(s.index(x),s.index(y))
    j=max(s.index(x),s.index(y))
    jj=0.5*(dist(x,y,M,s)+s_dist(x,y,M,s)-s_dist(y,x,M,s))
    hh=0.5*(dist(x,y,M,s)+s_dist(y,x,M,s)-s_dist(x,y,M,s))
    for letter in x:
        deep[letter]=deep[letter]+0.5*(dist(x,y,M,s)+s_dist(x,y,M,s)-s_dist(y,x,M,s))
    for letter in y:
        deep[letter]=deep[letter]+0.5*(dist(x,y,M,s)+s_dist(y,x,M,s)-s_dist(x,y,M,s))
    encoding[x+y]='('+str(encoding[x])+':'+str(0.5*(dist(x,y,M,s)+s_dist(x,y,M,s)-s_dist(y,x,M,s)))+', '+str(encoding[y])+':'+str(0.5*(dist(x,y,M,s)+s_dist(y,x,M,s)-s_dist(x,y,M,s)))+')'
    uu=[]
    for r in s:
        if r!=x and r!=y: 
            u=0.5*(dist(r,x,M,s)+dist(r,y,M,s)-dist(x,y,M,s))
            uu.append(u)
    s.pop(j)
    s.pop(i)
    s.append(x+y)
    uu.append(np.nan)
    M=np.delete(M, j, 0)
    M=np.delete(M, i, 0)
    M=np.delete(M, j, 1)
    M=np.delete(M, i, 1)
    M=np.insert(M, n-2, np.nan, axis=0)
    M=np.insert(M, n-2, uu, axis=1)
    return M, s, encoding,deep

def main(s,M):
    encoding={}
    deep={}
    for ss in s:
        encoding[ss]=ss
        deep[ss]=0
    while len(s)>2:
        x,y=ch(M,s)
        M,s,encoding,deep=step2(M,s,x,y,encoding,deep)
    aa=[]
    for l in s[0]:
        aa.append(deep[l])
    bb=[]
    for l in s[1]:
        bb.append(deep[l])
    e=M[0,1]+min(aa)+min(bb)
    if e/2-min(aa)>0 and e/2-min(bb)>0:
        t='('+encoding[s[0]]+':'+str(e/2-min(aa))+', '+encoding[s[1]]+':'+str(e/2-min(bb))+')'
    else:
        t='('+encoding[s[0]]+':'+str(M[0,1]/2)+', '+encoding[s[1]]+':'+str(M[0,1]/2)+')'
    print(t)


    

s=['A','B','C','D']
M = np.array([[np.nan,16,16,10],[np.nan, np.nan,8,8],[np.nan, np.nan,np.nan,4],[np.nan, np.nan,np.nan,np.nan]])   
main(s,M)
print()

s=['A','B','C','D','E','F']
M = np.array([[np.nan,5,4,7,6,8],[np.nan, np.nan,7,10,9,11],[np.nan, np.nan,np.nan,7,6,8],
              [np.nan, np.nan,np.nan,np.nan,5,9],[np.nan, np.nan,np.nan,np.nan,np.nan,8],
              [np.nan, np.nan,np.nan,np.nan,np.nan,np.nan]])
main(s,M)


(A:5.75, (B:5.5, (C:3.5, D:0.5):0.5):4.75)

(F:4.0, ((C:2.0, (A:1.0, B:4.0):1.0):1.0, (D:3.0, E:2.0):1.0):1.0)
